In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
sns.set_style("whitegrid")

In [3]:
%matplotlib inline

In [48]:
!ls 'data/'

spambase.DOCUMENTATION spambase.data          spambase.names         spambase.zip


In [170]:
import re
with open('data/spambase.names') as names:
    columns = names.readlines()

headers = []
for line in columns:
    q = re.match(r'([a-zA-Z]+_[a-zA-Z]+_)(?P<het>.*):', line)
    if q:
        headers.append(q.group('het'))
headers.append('IS_SPAM')

In [174]:
# read: data
words = pd.read_csv('data/spambase.data', names=headers)
words.head()

,make,address,all,3d,our,over,remove,internet,order,mail,...,;,(,[,!,$,#,length_average,length_longest,length_total,IS_SPAM
0,0.00,0.64,0.64,0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0,0.135,0.000,0.000,3.537,40,191,1


###Delete columns unrequired for regression

In [ ]:
del words['length_longest']
del words['length_average']
del words['length_total']

###Explicitly mark spam and non-spam

In [ ]:
spam = words['IS_SPAM'] == 1
non_spam = words['IS_SPAM'] == 0
words['IS_SPAM'][spam] = 'JUNK'
words['IS_SPAM'][non_spam] = 'NOJUNK'

In [176]:
words.head()

,make,address,all,3d,our,over,remove,internet,order,mail,...,edu,table,conference,;,(,[,!,$,#,IS_SPAM
0,0.00,0.64,0.64,0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0,0,0.00,0.000,0,0.778,0.000,0.000,JUNK
1,0.21,0.28,0.50,0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0,0,0.00,0.132,0,0.372,0.180,0.048,JUNK
2,0.06,0.00,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.06,0,0,0.01,0.143,0,0.276,0.184,0.010,JUNK
3,0.00,0.00,0.00,0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0,0,0.00,0.137,0,0.137,0.000,0.000,JUNK
4,0.00,0.00,0.00,0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0,0,0.00,0.135,0,0.135,0.000,0.000,JUNK


###Get the spam/nospam column out

In [177]:
junkno = words.pop('IS_SPAM')

###Split between training and test data

In [180]:
ytrain, ytest, xtrain, xtest = train_test_split(words, junkno, test_size=.40,)

In [184]:
xtest.head()

1994    NOJUNK
4391    NOJUNK
717       JUNK
2020    NOJUNK
3245    NOJUNK
Name: IS_SPAM, dtype: object

In [204]:
from sklearn.naive_bayes import MultinomialNB

In [205]:
classifier = MultinomialNB()

In [206]:
classifier.fit(ytrain, xtrain)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [209]:
classifier.score(ytrain,xtrain)

0.87644927536231887

In [212]:
classifier.score(ytest,xtest)

0.86746333514394347

In [213]:
xtest.head()

1994    NOJUNK
4391    NOJUNK
717       JUNK
2020    NOJUNK
3245    NOJUNK
Name: IS_SPAM, dtype: object

In [221]:
type(ytest)

pandas.core.frame.DataFrame

###Testing

In [222]:
classifier.predict(ytest.loc[717])

array(['JUNK'], 
      dtype='<U6')

In [223]:
classifier.predict(ytest.loc[3245])

array(['NOJUNK'], 
      dtype='<U6')